<a href="https://colab.research.google.com/github/bisenrose/Bose-Customer-Service-Chatbot/blob/main/bose_customer_service_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install tiktoken
!pip install langchain
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: packag

In [ ]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.9 MB/s eta 0:00:00


In [ ]:
import openai
import os
# Set up your OpenAI API key
os.environ['OPENAI_API_KEY'] = '----'
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/Bose Troubleshooting.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=50)
documents = text_splitter.split_documents(pages)

In [ ]:
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    llm=llm,
    output_key='answer',
    memory_key='chat_history',
    return_messages=True)

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "include_metadata": True})

prompt_template = """

You are a Customer Service Chatbot that has knowledge on music systems and help people by giving them clear, consice and simple systemic answers towards resolving their issues on the related topic.
If you do not know the answer guide the user to seek help from an offline service center.
Context: \n {context}?\n
question: \n {question} \n

Answer:

"""
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

messages = [
            SystemMessagePromptTemplate.from_template(prompt_template),
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

In [ ]:
chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  memory=memory,
  chain_type="stuff",
  retriever=retriever,
  return_source_documents=True,
  get_chat_history=lambda h : h,
  combine_docs_chain_kwargs={'prompt': qa_prompt},
  verbose=False)

In [ ]:
chat_history=[]

In [ ]:
query = "Hi there, how are you?"
result = chain({"question": query, "chat_history": chat_history})
result['answer']

In [ ]:
query = "I cannot skip forward or backward when playing a music service, what should i do?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

In [ ]:
!pip install streamlit streamlit_chat
!npm install localtunnel

In [ ]:
%%writefile app.py

import streamlit as st
import openai
import os
# Set up your OpenAI API key
os.environ['OPENAI_API_KEY'] = 'OPENAI_KEY'
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain import PromptTemplate
from streamlit_chat import message

st.set_page_config(page_title="NutriBot", page_icon=":smile:")

st.title('Welcome to (idk give it some name) bot!')

tab1= st.tabs(["📈 Talk Here"])

# from langchain.document_loaders import TextLoader  #for textfiles
# loader = TextLoader('/content/Bose Troubleshooting.pdf')
# documents = loader.load()

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/Bose Troubleshooting.pdf")
pages = loader.load_and_split()

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=50)
documents = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')

from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    llm=llm,
    output_key='answer',
    memory_key='chat_history',
    return_messages=True)

retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "include_metadata": True})

prompt_template = """

You are a Customer Service Chatbot that has knowledge on music systems and help people by giving them clear, consice and simple systemic answers towards resolving their issues on the related topic.
If you do not know the answer guide the user to seek help from an offline service center.
If the query is outside this domain then do not answer it.
Context: \n {context}?\n
question: \n {question} \n

Answer:

 """
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

messages = [
            SystemMessagePromptTemplate.from_template(prompt_template),
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  memory=memory,
  chain_type="stuff",
  retriever=retriever,
  return_source_documents=True,
  get_chat_history=lambda h : h,
  combine_docs_chain_kwargs={'prompt': qa_prompt},
  verbose=False)

if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = []
if "generated" not in st.session_state:
    st.session_state["generated"] = ["Hello ! Ask me anything about Bose Music Systems!"]
if "past" not in st.session_state:
    st.session_state["past"] = ["Hey ! 👋"]

#container for the chat history
response_container = st.container()
#container for the user's text input
container = st.container()

def generate_response(query):
    result = chain({"question": query, "chat_history": st.session_state["chat_history"]})
    #st.write(result['answer'])
    st.session_state["chat_history"] = [(query, result["answer"])]
    with st.sidebar:
        st.write(result['source_documents'])
    return result["answer"]

with container:
  with st.form(key="my_form", clear_on_submit=True):
      user_input = st.text_input("You:", key="input")
      submit_button = st.form_submit_button(label="Send")

      if user_input and submit_button:
        output = generate_response(user_input)
        #print(output)
        st.session_state["past"].append(user_input)
        st.session_state["generated"].append(output)
        st.session_state["chat_history"] = [(user_input, output)]

with response_container:
    for i in range(len(st.session_state['generated'])):
        message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="adventurer")
        message(st.session_state["generated"][i], key=str(i))


In [ ]:
!curl https://ipv4.icanhazip.com/

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501